In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import functools
import time
import os
import pandas as pd
import csv
import sys

%tensorflow_version 1.x 
import numpy as np
import scipy.sparse as sparse
from sklearn.decomposition import non_negative_factorization
import tensorflow as tf
import tensorflow_probability as tfp

tf.reset_default_graph() #to resue tf variable 

#from sklearn.decomposition import NMF, non_negative_factorization 
tf.test.gpu_device_name()

TensorFlow 1.x selected.


'/device:GPU:0'

In [ ]:
#parameters
num_topics = 25
max_steps = 300000
print_steps = 25000
 
## parameters for ADAM optimizer
eps = 1
learningrate = 0.0001
batch_size = 512

path = '/content/gdrive/My Drive/tbip_BSP/data/'
#path to tbip.py, to change as needed
py_file = '/content/gdrive/My Drive/tbip_BSP/code/'  
#vocabulary
vocab = pd.read_csv(os.path.join(path,'vocabulary.txt'), header = None)
vocabulary = vocab[0].tolist()

In [ ]:
sys.path.append(os.path.abspath(py_file))
import tbip

In [ ]:
#to start from session 98 onwards
for i in range(111, 115):
  tf.reset_default_graph() #to be used if graph building doesn't renew 
  tf.set_random_seed(0) #after tf_reset_graph
  random_state = np.random.RandomState(0) #after tf_reset_graph
  old_dir = os.path.join(path, str(i-1), 'output' + str(max_steps))
  data_dir = os.path.join(path, str(i), 'input')
  save_dir = os.path.join(path, str(i), 'output' + str(max_steps))  # ~/data/<sess>/output

  if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

  objective_topic_loc = np.load(os.path.join(old_dir, "objective_topic_loc.npy"))
  beta = objective_topic_loc
  beta = np.exp(beta)

  X = sparse.load_npz(os.path.join(data_dir, 'counts.npz')) 
  #X = X.astype('float64')
  W, H, n_iter = non_negative_factorization(X, n_components = num_topics, 
                                            init='custom', 
                                            random_state=0, 
                                            update_H = False, 
                                            H = beta.astype('float32')) #paul:we had astype('double') but returned error
  
  W = W.astype('float32')
  W = W + 1e-3 #causes 0 value otherwise
  initial_document_loc = W #theta
  objective_topic_loc_prev = beta.astype('float32') #don't need, use objective_topic_loc

  ideological_topic_loc_prev = np.load(os.path.join(old_dir, 'ideological_topic_loc.npy')) #eta
  ideological_topic_loc_prev = tf.convert_to_tensor(ideological_topic_loc_prev)

  #using tbip.build_input_pipeline
  #original source code available at: https://github.com/keyonvafa/tbip
  (iterator, author_weights, vocabulary, author_map, 
  num_documents, num_words, num_authors) = tbip.build_input_pipeline(
        data_dir, 
        batch_size,
        random_state,
        counts_transformation='nothing')
  document_indices, counts, author_indices = iterator.get_next()

  # Create Lognormal variational family for document intensities (theta).
  document_loc = tf.get_variable(
      "document_loc",
      initializer=tf.constant(np.log(initial_document_loc))) #theta from NMF

  document_scale_logit = tf.get_variable(
      "document_scale_logit",
      shape=[num_documents, num_topics],
      initializer=tf.initializers.random_normal(mean=-2, stddev=1.),
      dtype=tf.float32)

  document_scale = tf.nn.softplus(document_scale_logit)

  document_distribution = tfp.distributions.LogNormal(
      loc=document_loc,
      scale=document_scale) 

  # Create Lognormal variational family for objective topics (beta).

  objective_topic_loc = tf.get_variable(
    "objective_topic_loc",
    initializer=tf.constant(np.log(objective_topic_loc_prev)), #from session 97
    trainable = True)
  
  objective_topic_scale_logit = tf.get_variable(
      "objective_topic_scale_logit",
      shape=[num_topics, num_words],
      initializer=tf.initializers.random_normal(mean=-2, stddev=1.),
      dtype=tf.float32,
      trainable = True)
  
  objective_topic_scale = tf.nn.softplus(objective_topic_scale_logit)

  objective_topic_distribution = tfp.distributions.LogNormal(
      loc=objective_topic_loc,
      scale=objective_topic_scale)
    
  # Create Gaussian variational family for ideological topics (eta).
  
  ideological_topic_loc = tf.get_variable(
    "ideological_topic_loc",
    initializer = ideological_topic_loc_prev, #initialized only ideological_topic_loc so that var type doesn't change
    trainable = True)
  
  ideological_topic_scale_logit = tf.get_variable(
      "ideological_topic_scale_logit",
      shape=[num_topics, num_words],
      dtype=tf.float32,
      trainable = True)
  
  ideological_topic_scale = tf.nn.softplus(ideological_topic_scale_logit)

  ideological_topic_distribution = tfp.distributions.Normal(
      loc=ideological_topic_loc,
      scale=ideological_topic_scale)

  # Create Gaussian variational family for ideal points (x).
  ideal_point_loc = tf.get_variable(
      "ideal_point_loc",
      shape=[num_authors],
      dtype=tf.float32)

  ideal_point_scale_logit = tf.get_variable(
      "ideal_point_scale_logit",
      initializer=tf.initializers.random_normal(mean=0, stddev=1.),
      shape=[num_authors],
      dtype=tf.float32)

  ideal_point_scale = tf.nn.softplus(ideal_point_scale_logit)

  ideal_point_distribution = tfp.distributions.Normal(
      loc=ideal_point_loc,
      scale=ideal_point_scale)

  # Approximate ELBO.
  elbo = tbip.get_elbo(counts,
                      document_indices,
                      author_indices,
                      author_weights,
                      document_distribution,
                      objective_topic_distribution, 
                      ideological_topic_distribution,
                      ideal_point_distribution,
                      num_documents,
                      batch_size)
  loss = -elbo


  optim = tf.train.AdamOptimizer(learning_rate=learningrate, epsilon=eps)

  #original_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=1e-04)
  #optimizer = tf.contrib.estimator.clip_gradients_by_norm(original_optimizer, clip_norm=5.0)
  train_op = optim.minimize(loss)

  document_mean = document_loc + document_scale ** 2 / 2 

  neutral_mean = objective_topic_loc + objective_topic_scale ** 2 / 2

  positive_mean = (objective_topic_loc + 
                  ideological_topic_loc + 
                  (objective_topic_scale ** 2 + 
                    ideological_topic_scale ** 2) / 2) #intialized
                    

  negative_mean = (objective_topic_loc - 
                  ideological_topic_loc +
                  (objective_topic_scale ** 2 + 
                    ideological_topic_scale ** 2) / 2) #intialized
  
  print('Learning for session ' + str(i))
  
  loss_vals = []
  init = tf.global_variables_initializer()
  sess = tf.Session()
  sess.run(init)
  start_time = time.time()
  for step in range(max_steps):
    (_, elbo_val) = sess.run([train_op, elbo])
    loss_vals.append(elbo_val)
    duration = (time.time() - start_time) / (step + 1)
    if step % print_steps == 0 or step == max_steps - 1:
      print("Step: {:>3d} ELBO: {:.3f} ({:.3f} sec/step)".format(
          step, elbo_val, duration))
    if (step + 1) % 50000 == 0 or step == max_steps - 1:
      (document_topic_mean, neutral_topic_mean, negative_topic_mean, positive_topic_mean, 
     ideal_point_mean, ots, otl, itl, its) = sess.run([document_mean, neutral_mean, negative_mean, 
                                   positive_mean, ideal_point_loc, objective_topic_scale, 
                                   objective_topic_loc, 
                                   ideological_topic_loc, 
                                   ideological_topic_scale])
      np.save(os.path.join(save_dir, "document_topic_mean.npy"), document_topic_mean)
      np.save(os.path.join(save_dir, "neutral_topic_mean.npy"), neutral_topic_mean)
      np.save(os.path.join(save_dir, "negative_topic_mean.npy"), negative_topic_mean)
      np.save(os.path.join(save_dir, "positive_topic_mean.npy"), positive_topic_mean)
      np.save(os.path.join(save_dir, "ideal_point_mean.npy"), ideal_point_mean)
      np.save(os.path.join(save_dir, "objective_topic_scale.npy"), ots)
      np.save(os.path.join(save_dir, "objective_topic_loc.npy"), otl)
      np.save(os.path.join(save_dir, "ideological_topic_loc.npy"), itl)
      np.save(os.path.join(save_dir, "ideological_topic_scale.npy"), its)        
      loss_df = pd.DataFrame(data={"loss": loss_vals}) 
      loss_df.to_csv(os.path.join(save_dir, 'loss_values.csv'), sep=',', index=False) #save loss values as a csv file

  print("session " + str(i) + " done!")
  #i = i + 1

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Learning for session 111
Step:   0 ELBO: -6307892.000 (1.679 sec/step)
Step: 25000 ELBO: -5322009.000 (0.058 sec/step)
Step: 50000 ELBO: -4721635.000 (0.058 sec/step)
Step: 75000 ELBO: -3889032.000 (0.058 sec/step)
Step: 100000 ELBO: -3658967.250 (0.058 sec/step)
Step: 125000 ELBO: -3547457.500 (0.058 sec/step)
Step: 150000 ELBO: -3504370.500 (0.058 sec/step)
Step: 175000 ELBO: -3726771.000 (0.058 sec/step)
Step: 200000 ELBO: -3227716.250 (0.058 sec/step)
Step: 225000 ELBO: -3448802.000 (0.058 sec/step)
Step: 250000 ELBO: -3322487.500 (0.058 sec/step)
Step: 275000 ELBO: -3384215.000 (0.058 sec/step)
Step: 299999 ELBO: -3476856.750 (0.058 sec/step